# Задание 4. Реализовать рекомендательную систему по подбору пива на основе  датасета «BeerAdvocate»

## Downloading and unzipping data

In [ ]:
# Change directory for kaggle JSON
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks")

In [ ]:
# Create a kaggle folder
!mkdir -p ~/.kaggle

# Copy kaggle.json to created folder
!cp kaggle.json ~/.kaggle/

In [ ]:
os.chdir("/content")

In [ ]:
# Permission for the json to act
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download the required dataset
!kaggle datasets download -d thedevastator/1-5-million-beer-reviews-from-beer-advocate

 28% 9.00M/32.5M [00:00<00:00, 37.8MB/s]
100% 32.5M/32.5M [00:00<00:00, 100MB/s] 


In [ ]:
# Unzip our dataset
from zipfile import ZipFile
from tqdm import tqdm


file_to_extract = "1-5-million-beer-reviews-from-beer-advocate.zip"

# Open your .zip file
with ZipFile(file=file_to_extract) as zip_file:

    # Loop over each file and extract them
    for file in tqdm(iterable=zip_file.namelist(), total=len(zip_file.namelist())):
        zip_file.extract(member=file)

100%|██████████| 1/1 [00:01<00:00,  1.26s/it]


## Data Preprocessing

In [ ]:
import pandas as pd
import polars as pl
import time

In [ ]:
start_time = time.time()
df = pd.read_csv("/content/beer_reviews.csv", index_col="index")
end_time = time.time()

print(f"Elapsed time to read csv-file is: {end_time - start_time}")
df.head()

Elapsed time to read csv-file is: 4.048144817352295


,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
index,,,,,,,,,,,,,
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [ ]:
start_time = time.time()
df = pl.read_csv("/content/beer_reviews.csv")
end_time = time.time()

print(f"Elapsed time to read csv-file is: {end_time - start_time}")
df.head()

Elapsed time to read csv-file is: 1.9511523246765137


index,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
i64,i64,str,i64,f64,f64,f64,str,str,f64,f64,str,f64,i64
0,10325,"""Vecchio Birrai…",1234817823,1.5,2.0,2.5,"""stcules""","""Hefeweizen""",1.5,1.5,"""Sausa Weizen""",5.0,47986
1,10325,"""Vecchio Birrai…",1235915097,3.0,2.5,3.0,"""stcules""","""English Strong…",3.0,3.0,"""Red Moon""",6.2,48213
2,10325,"""Vecchio Birrai…",1235916604,3.0,2.5,3.0,"""stcules""","""Foreign / Expo…",3.0,3.0,"""Black Horse Bl…",6.5,48215
3,10325,"""Vecchio Birrai…",1234725145,3.0,3.0,3.5,"""stcules""","""German Pilsene…",2.5,3.0,"""Sausa Pils""",5.0,47969
4,1075,"""Caldera Brewin…",1293735206,4.0,4.5,4.0,"""johnmichaelsen…","""American Doubl…",4.0,4.5,"""Cauldron DIPA""",7.7,64883
